## 08 Classifying various hair types with Deep Neural Networks

### 8.1 Setup and Data Preparation

In [11]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

Torch version: 2.9.0+cu126
CUDA available: True
Device: Tesla T4


In [14]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [15]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [12]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

--2025-12-01 08:46:33--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T09%3A27%3A06Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T08%3A26%3A09Z&ske=2025-12-01T09%3A27%3A06Z&sks=b&skv=2018-11-09&sig=dDXpRJe40sOK1Sb6CnGl6thPaBb37c6Nx3mkQLKMiQM%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDU4MDU5MywibmJmIjoxNzY0NTc4NzkzLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG

In [16]:
!unzip data.zip -d data

Archive:  data.zip
   creating: data/data/
   creating: data/data/test/
   creating: data/data/test/curly/
  inflating: data/data/test/curly/03312ac556a7d003f7570657f80392c34.jpg  
  inflating: data/data/test/curly/106dfcf4abe76990b585b2fc2e3c9f884.jpg  
  inflating: data/data/test/curly/1a9dbe23a0d95f1c292625960e4509184.jpg  
  inflating: data/data/test/curly/341ea26e6677b655f8447af56073204a4.jpg  
  inflating: data/data/test/curly/61aPFVrm42L._SL1352_.jpg  
  inflating: data/data/test/curly/6d8acb0fe980774ea4e5631198587f45.png  
  inflating: data/data/test/curly/7f5649a0c33a2b334f23221a52c16b9b.jpg  
  inflating: data/data/test/curly/90146673.jpg  
  inflating: data/data/test/curly/9b3608e01d78fbabc9fb0719323d507f4.jpg  
  inflating: data/data/test/curly/b171c99161f3cffc12d4b74488ef2fc6.jpg  
  inflating: data/data/test/curly/blogger_one.jpg  
  inflating: data/data/test/curly/c03ca1590aa4df74e922ad8257305a2b.jpg  
  inflating: data/data/test/curly/c1b89bb4f86a3478ec20ce1f63f003c1.jp

In [18]:
!ls -R .

.:
data  data.zip	main.py  pyproject.toml  README.md  sample_data  uv.lock

./data:
data

./data/data:
test  train

./data/data/test:
curly  straight

./data/data/test/curly:
 03312ac556a7d003f7570657f80392c34.jpg
 106dfcf4abe76990b585b2fc2e3c9f884.jpg
 1a9dbe23a0d95f1c292625960e4509184.jpg
 341ea26e6677b655f8447af56073204a4.jpg
 61aPFVrm42L._SL1352_.jpg
 6d8acb0fe980774ea4e5631198587f45.png
 7f5649a0c33a2b334f23221a52c16b9b.jpg
 90146673.jpg
 9b3608e01d78fbabc9fb0719323d507f4.jpg
 b171c99161f3cffc12d4b74488ef2fc6.jpg
 blogger_one.jpg
 c03ca1590aa4df74e922ad8257305a2b.jpg
 c1b89bb4f86a3478ec20ce1f63f003c1.jpg
 c5.jpg
 C86_76156_405725089516021_1474585421_n.jpg
 CG-Divya2.jpg
 Curls-1600x900.jpg
 Curly20Hair20-20Tout.jpg
 Curly-hair_1024x400.jpg
 curly-hair-412.jpg
 curly-hair-model_18028937ed8_medium.jpg
 curly-hairstyles-for-teenage-girls.jpg
 cury-hair-4dec18-default.jpg
 Dense-Curly-Hair-for-Indian-Girls.jpg
 ecbc8abdd63850d81fa5b3d0ba92ab96.jpg
 f30986c3b61473bc1153884a808963af.jpg

In [20]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Base Folder
base_dir = "data/data"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

print("Training data directory:", train_dir)
print("Testing data directory:", test_dir)

image_size = (200, 200)

train_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

Training data directory: data/data/train
Testing data directory: data/data/test


In [21]:
from torchvision import datasets
from torch.utils.data import DataLoader

batch_size = 20

# Datasets
train_dataset = datasets.ImageFolder(
    root=train_dir,
    transform=train_transforms
)

test_dataset = datasets.ImageFolder(
    root=test_dir,
    transform=test_transforms
)

print("Classes:", train_dataset.classes)
print("Class to index:", train_dataset.class_to_idx)
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

# Dataloaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
)

Classes: ['curly', 'straight']
Class to index: {'curly': 0, 'straight': 1}
Train size: 800
Test size: 201


### 8.2 Setting Up Convolutional Neural Network (CNN) Model

In [27]:
import torch.nn as nn
import torch.nn.functional as F

class HairClassifierCNN(nn.Module):
    def __init__(self):
        super(HairClassifierCNN, self).__init__()

        # feature extractor
        self.conv1 = nn.Conv2d(
            in_channels=3, 
            out_channels=32, 
            kernel_size=3,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # After conv + pool, image size changes:
        # Input: (3, 200, 200)
        # Conv1: (32, 198, 198)
        # Pool1: (32, 99, 99)
        self.flatten_dim = 32 * 99 * 99

        # classifier
        self.fc1 = nn.Linear(self.flatten_dim, 64)
        self.fc2 = nn.Linear(64, 1) # Final input: one logit

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, self.flatten_dim)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [28]:
model = HairClassifierCNN().to(device)
print(model)

HairClassifierCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


### 8.3 Define loss function: Using Binary Cross-Entropy Loss

In [29]:
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.002,
    momentum=0.8
)

print("Setup complete. Using: BCEWithLogitsLoss and SGD optimizer.")

Setup complete. Using: BCEWithLogitsLoss and SGD optimizer.


#### Summary of the model. For Q2

In [30]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


### 8.4 Generator and Tranining

In [31]:
import torch
import numpy as np

num_epochs = 10

history = {
    "loss": [],
    "acc": [],
    "val_loss": [],
    "val_acc": [],
}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)  # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()

        # forward 
        output = model(images)
        loss = criterion(output, labels)

        # backward and optimize
        loss.backward()
        optimizer.step()

        # Stats
        running_loss += loss.item() * images.size(0)
        
        # predictions: apply sigmoid then threshold at 0.5
        predicted = (torch.sigmoid(output) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_train
    epoch_acc = correct_train / total_train
    history["loss"].append(epoch_loss)
    history["acc"].append(epoch_acc)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0


    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)  # Ensure labels are float and have shape (batch_size, 1)
    
            output = model(images)
            loss = criterion(output, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(output)> 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / total_val
    val_epoch_acc = correct_val / total_val
    history["val_loss"].append(val_epoch_loss)
    history["val_acc"].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6382, Acc: 0.6362, Val Loss: 0.6298, Val Acc: 0.6169
Epoch 2/10, Loss: 0.5375, Acc: 0.7312, Val Loss: 0.5968, Val Acc: 0.6617
Epoch 3/10, Loss: 0.4899, Acc: 0.7412, Val Loss: 0.5997, Val Acc: 0.6567
Epoch 4/10, Loss: 0.4761, Acc: 0.7662, Val Loss: 0.6087, Val Acc: 0.6716
Epoch 5/10, Loss: 0.4025, Acc: 0.8213, Val Loss: 0.6044, Val Acc: 0.6866
Epoch 6/10, Loss: 0.3731, Acc: 0.8387, Val Loss: 0.6541, Val Acc: 0.6816
Epoch 7/10, Loss: 0.2638, Acc: 0.8888, Val Loss: 0.6605, Val Acc: 0.6965
Epoch 8/10, Loss: 0.2163, Acc: 0.9150, Val Loss: 0.7561, Val Acc: 0.6716
Epoch 9/10, Loss: 0.1772, Acc: 0.9350, Val Loss: 0.7618, Val Acc: 0.6965
Epoch 10/10, Loss: 0.1381, Acc: 0.9587, Val Loss: 0.7356, Val Acc: 0.7114


### 8.5 Data Augmentation

In [34]:
augmented_train_transfroms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [35]:
aug_train_dataset = datasets.ImageFolder(
    root=train_dir,
    transform=augmented_train_transfroms
)

aug_train_loader = DataLoader(
    aug_train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

print("Augmented training dataset size:", len(aug_train_dataset))

Augmented training dataset size: 800


In [36]:
import torch
import numpy as np

num_epochs_aug = 10
aug_val_losses = []
aug_val_accs = []

for epoch in range(num_epochs_aug):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in aug_train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)  # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()

        # forward 
        output = model(images)
        loss = criterion(output, labels)

        # backward and optimize
        loss.backward()
        optimizer.step()

        # Stats
        running_loss += loss.item() * images.size(0)
        
        # predictions: apply sigmoid then threshold at 0.5
        predicted = (torch.sigmoid(output) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / total_train
    train_acc = correct_train / total_train

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0


    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)  # Ensure labels are float and have shape (batch_size, 1)
    
            output = model(images)
            loss = criterion(output, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(output)> 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = val_running_loss / total_val
    val_acc = correct_val / total_val

    aug_val_losses.append(val_loss)
    aug_val_accs.append(val_acc)
    print(f"Aug Epoch {epoch+1}/{num_epochs_aug}, "
          f"- train_acc: {train_acc:.4f}, val_acc: {val_acc:.4f}, val_loss: {val_loss:.4f}")

Aug Epoch 1/10, - train_acc: 0.6737, val_acc: 0.6169, val_loss: 0.6590
Aug Epoch 2/10, - train_acc: 0.7225, val_acc: 0.6965, val_loss: 0.5848
Aug Epoch 3/10, - train_acc: 0.7462, val_acc: 0.6965, val_loss: 0.5807
Aug Epoch 4/10, - train_acc: 0.7612, val_acc: 0.6418, val_loss: 0.6951
Aug Epoch 5/10, - train_acc: 0.7325, val_acc: 0.7363, val_loss: 0.5762
Aug Epoch 6/10, - train_acc: 0.7562, val_acc: 0.7015, val_loss: 0.5739
Aug Epoch 7/10, - train_acc: 0.7700, val_acc: 0.6219, val_loss: 0.7227
Aug Epoch 8/10, - train_acc: 0.7688, val_acc: 0.7015, val_loss: 0.5843
Aug Epoch 9/10, - train_acc: 0.8125, val_acc: 0.7512, val_loss: 0.5090
Aug Epoch 10/10, - train_acc: 0.8087, val_acc: 0.7562, val_loss: 0.5139


In [37]:
!nvidia-smi

Mon Dec  1 10:14:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             33W /   70W |     896MiB /  15360MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 8.6 Calculations for Questions

#### For Q3 and Q4

In [38]:
import numpy as np

train_acc_array = np.array(history["acc"])
train_loss_array = np.array(history["loss"])

q3_median_acc = np.median(train_acc_array)
q4_std_loss = np.std(train_loss_array)

q3_median_acc, q4_std_loss

(np.float64(0.8300000000000001), np.float64(0.15892079134051343))

#### For Q5 and Q6

In [39]:
aug_losses = np.array(aug_val_losses)
aug_accs = np.array(aug_val_accs)

q5_mean_loss = np.mean(aug_losses)
q6_last5_mean_acc = np.mean(aug_accs[-5:])

q5_mean_loss, q6_last5_mean_acc

(np.float64(0.5999647927373202), np.float64(0.7064676616915423))